<a href="https://colab.research.google.com/github/hallpaz/3dsystems21/blob/main/assignments/LabClass2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPA 2021 - 3D Graphics Systems: AI Graphics
## A demo of some functionalities of PyTorch3D

In [ ]:
!pip install torch torchvision
import os
import sys
import torch
if torch.__version__=='1.6.0+cu101' and sys.platform.startswith('linux'):
    !pip install pytorch3d
else:
    need_pytorch3d=False
    try:
        import pytorch3d
    except ModuleNotFoundError:
        need_pytorch3d=True
    if need_pytorch3d:
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

## Data Structures

In [ ]:
import torch
from pytorch3d.structures import Meshes

In [ ]:
# Setup
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

In [ ]:
!wget -P . https://raw.githubusercontent.com/hallpaz/3dsystems20/master/extensions_utils/cube.py

In [ ]:
# Vertex coordinates for a level 0 cube.
_cube_verts0 = [
    [-0.50, 0.50, 0.50],
    [-0.50, -0.50, 0.50],
    [0.50, -0.50, 0.50],
    [0.50, 0.50, 0.50],

    [-0.50, 0.50, -0.50],
    [-0.50, -0.50, -0.50],
    [0.50, -0.50, -0.50],
    [0.50, 0.50, -0.50]
]


# Faces for level 0 cube
_cube_faces0 = [
    [0, 1, 2],
    [2, 3, 0],
    [7, 6, 5],
    [4, 7, 5],
    [6, 3, 2],
    [3, 6, 7],
    [4, 5, 0],
    [0, 5, 1],
    [3, 4, 0],
    [4, 3, 7],
    [2, 1, 5],
    [5, 6, 2],
]

In [ ]:
from cube import cube

In [ ]:
refinedcube = cube(1, device=device)

In [ ]:
verts_list = [torch.tensor(_cube_verts0, device=device), refinedcube.verts_list()[0]]
faces_list = [torch.tensor(_cube_faces0, dtype=torch.int64, device=device), refinedcube.faces_list()[0]]

mesh_batch = Meshes(verts=verts_list, faces=faces_list)

## Packed and Padded Tensors

In [ ]:
# packed representation
verts_packed = mesh_batch.verts_packed()

# auxiliary tensors
mesh_to_vert_idx = mesh_batch.mesh_to_verts_packed_first_idx()
vert_to_mesh_idx = mesh_batch.verts_packed_to_mesh_idx()

# edges
edges = mesh_batch.edges_packed()

# face normals
face_normals = mesh_batch.faces_normals_packed()

In [ ]:
verts_packed

In [ ]:
mesh_batch.verts_padded()

## Input / Output

In [ ]:
!mkdir -p data
!wget -P data https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.obj
!wget -P data https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow.mtl
!wget -P data https://dl.fbaipublicfiles.com/pytorch3d/data/cow_mesh/cow_texture.png

In [ ]:
from pytorch3d.io import load_obj

In [ ]:
obj_file = "data/cow.obj"
verts, faces, aux = load_obj(obj_file)

faces = faces.verts_idx
normals = aux.normals
textures = aux.verts_uvs
materials = aux.material_colors
tex_maps = aux.texture_images

In [ ]:
tex_maps

In [ ]:
import matplotlib.pyplot as plt
from pytorch3d.renderer import Textures

In [ ]:
plt.imshow(tex_maps['material_1'])

# 3D Transforms

In [ ]:
from pytorch3d.transforms import Transform3d, Rotate, Translate

In [ ]:
# example 1
T = Translate(torch.FloatTensor([[1.0, 2.0, 3.0]]), device=device)
R = Rotate(torch.FloatTensor([[0, 1, 0], [0, 0, 1], [1, 0, 0]]), device=device)
RT = Transform3d(device=device).compose(R, T)

In [ ]:
RT.get_matrix()

In [ ]:
# applying Transform
verts_transformed = RT.transform_points(mesh_batch.verts_packed())
verts_transformed

# Renderer

In [ ]:
from pytorch3d.renderer import (
    OpenGLPerspectiveCameras, look_at_view_transform,
    RasterizationSettings, BlendParams,
    MeshRenderer, MeshRasterizer, HardPhongShader,
    Textures
)

In [ ]:
R, T = look_at_view_transform(2.7, 10, 20)
cameras = OpenGLPerspectiveCameras(device=device, R=R, T=T)
raster_settings = RasterizationSettings(
    image_size=512,
    blur_radius=0.0,
    faces_per_pixel=1, # sets the value of K
)

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
    shader=HardPhongShader(device=device, cameras=cameras)
)

In [ ]:
# Creating a texture for the mesh
white_tex = torch.ones_like(mesh_batch.verts_packed())[None]
textures = Textures(verts_rgb=white_tex.to(device))
mesh_batch.textures = textures

In [ ]:
images = renderer(mesh_batch, cameras=cameras)

In [ ]:
def plot_side_by_side(images):
  n = images.shape[0]
  fig = plt.figure(figsize=(10, 10))
  for i in range(n):
    fig.add_subplot(1, n, i+1)
    plt.imshow(images[i, ..., :3].cpu().numpy())
    # plt.grid("off");
    # plt.axis("off");

In [ ]:
plot_side_by_side(images)

In [ ]:
from math import radians, cos, sin

In [ ]:
cos45 = cos(radians(45))
sin45 = sin(radians(45))
# applying a transform to the first mesh
SR = Transform3d(device=device).scale(1.0, 1.5, 1.0).rotate(
      R=torch.tensor([[cos45, -sin45, 0.0], [sin45, cos45, 0.0], [0.0, 0.0, 1.0]])
    )
verts0 = mesh_batch.verts_list()[0]
verts0 = SR.transform_points(verts0)
verts1 = mesh_batch.verts_list()[1]
mesh_batch2 = Meshes(verts=[verts0, verts1], faces=mesh_batch.faces_list(), textures=mesh_batch.textures)

In [ ]:
plot_side_by_side(renderer(mesh_batch2))